### Create AML SDK docker file and build and test image

To run this notebook, start it via a plain Jupyter notebook session on your Ubuntu_Azure_VM:

* login (ssh) into the VM:
```
loginname1@Ubuntu_Azure_VM:/datadrive01/prj/regularR_Realtime$ jupyter notebook --notebook-dir=$(pwd) --ip='*' --port=9000 --no-browser --allow-root
```
  
   
* then go to:    
http://Ubuntu_Azure_VM.eastus2.cloudapp.azure.com:9000/ 
  
  
https://docs.microsoft.com/en-us/azure/machine-learning/service/azure-machine-learning-release-notes  
2019-03-11
Azure Machine Learning SDK for Python v1.0.18  
   
Note: As of 02/13/2019, latest versions are:  
Python 3.7.2 (https://www.python.org/downloads/, https://anaconda.org/anaconda/python)  
miniconda3 4.5.12 (https://hub.docker.com/r/continuumio/miniconda3/tags)  
AML SDK  v.1.0.15 (https://docs.microsoft.com/en-us/python/api/overview/azure/ml/install?view=azure-ml-py)  
Ubuntu 19.04 (https://hub.docker.com/_/ubuntu?tab=tags)  

In [1]:
# Allow multiple displays per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [2]:
%load_ext dotenv
# import dotenv
# print (dir(dotenv))

In [3]:
%%writefile .env

# Your docker login and image repository name
docker_login = 'georgedockeraccount'
image_tag = "aml-sdk_docker_image"
image_version = 'sdk.v1.0.76' # also set this in conda file below

Overwriting .env


In [4]:
%dotenv
docker_file_location = 'docker_build'

import os
docker_file_name = 'dockerfile'+ '_' + os.getenv('image_tag') + '_' + os.getenv('image_version')
conda_dependency_file = 'aml_sdk_conda_dep_file.yml'

In [5]:
local_dir = !pwd
docker_file_dir = os.path.join(*([local_dir.s, docker_file_location]))
docker_file_dir
 
docker_file_path = os.path.join(*([docker_file_dir]+[docker_file_name]))
docker_file_path

conda_dependency_file_path = os.path.join(*([docker_file_dir]+[conda_dependency_file]))
conda_dependency_file_path

#clean previous history (i.e. content of amlsdk/docker)
!mkdir -p {docker_file_dir}
!chmod -R ugo=rwx $docker_file_dir
!rm -rf $docker_file_dir/*
!ls -l $docker_file_dir

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build'

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/dockerfile_aml-sdk_docker_image_sdk.v1.0.76'

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/aml_sdk_conda_dep_file.yml'

total 0


In [6]:
%%writefile $conda_dependency_file_path
name: aml_sdk_conda_env02
    
channels:
  - anaconda
dependencies:
  - python=3.7 # 3.6 req by tf, which seems to be req by automl, not 3.7.2 
  - numpy
  - cython
  - notebook 
  - nb_conda
  - scikit-learn
  - pip
  - pip:
    - python-dotenv
    - papermill[azure]
    - azureml-sdk[notebooks,automl,explain]==1.0.76 

Writing /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/aml_sdk_conda_dep_file.yml


In [7]:
%%writefile $docker_file_path 

# FROM continuumio/miniconda3:4.6.14 as AzureMLSDKOnly 
FROM continuumio/miniconda3:4.7.12 as AzureMLSDKOnly    
MAINTAINER George Iordanescu <ghiordan@microsoft.com>

RUN apt-get update --fix-missing && apt-get install -y --no-install-recommends \
    gpg-agent \
    gcc g++ \
    wget bzip2 \
    curl apt-transport-https lsb-release gpg \
    git make \
    sudo && \
    apt-get clean && \
    rm -rf /var/lib/apt/lists/*

ADD aml_sdk_conda_dep_file.yml /tmp/aml_sdk_conda_dep_file.yml
ENV CONDA_DIR /opt/conda
ENV CONDA_ENV_NAME aml-sdk-conda-env    

# reate aml sdk conda env defined by .yml file
RUN /opt/conda/bin/conda env create -q --name $CONDA_ENV_NAME -f /tmp/aml_sdk_conda_dep_file.yml && \
    /opt/conda/bin/conda clean  --yes --all 
ENV CONDA_AUTO_UPDATE_CONDA=false
ENV CONDA_DEFAULT_ENV=$CONDA_ENV_NAME
ENV CONDA_PREFIX=$CONDA_DIR/envs/$CONDA_DEFAULT_ENV
ENV PATH=$CONDA_PREFIX/bin:/opt/conda/bin:$PATH                
        
FROM AzureMLSDKOnly as AzureMLSDKAndAzCli
# Install Azure CLI
ENV AZ_CLI_REPO=stretch   
RUN echo "deb [arch=amd64] https://packages.microsoft.com/repos/azure-cli/ $AZ_CLI_REPO main" | \
    tee /etc/apt/sources.list.d/azure-cli.list && \
    curl -L https://packages.microsoft.com/keys/microsoft.asc | apt-key add - && \
    apt-get update && \
    apt-get install -y --no-install-recommends \
    azure-cli 
    
# Install AzCopy
RUN mkdir -p /tmp/azcopy && mkdir -p /azcopy10 && \
    wget -O /tmp/azcopy/azcopyv10.tar.gz https://aka.ms/downloadazcopy-v10-linux &&  \
    tar -xf /tmp/azcopy/azcopyv10.tar.gz -C /azcopy10 
RUN rm -rf /tmp/azcopy
ENV PATH=/azcopy10/azcopy_linux_amd64_10.3.2:$PATH
    
FROM AzureMLSDKAndAzCli as AzureMLSDKAndAzCliAndAZDevOps
#https://marketplace.visualstudio.com/items?itemName=ms-vsts.cli
RUN az extension add --name azure-devops
        
CMD /bin/bash


Writing /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/dockerfile_aml-sdk_docker_image_sdk.v1.0.76


In [8]:
%dotenv

#add docker image version info only if image_version has been set to a non-empty string, otherwise let docker increment it
image_version_string = os.getenv('image_version')
if image_version_string!="":
    image_version_string = ':'  + image_version_string
    
aml_docker_image_name = os.getenv('docker_login') + '/' + os.getenv('image_tag') + \
image_version_string
# aml_sudo_docker_image_name = os.getenv('docker_login') + '/' + os.getenv('image_tag') + \
# 'sudo' + image_version_string
aml_azcli_docker_image_name = os.getenv('docker_login') + '/' + os.getenv('image_tag') + \
'azcli' + image_version_string
aml_azcli_azdevops_docker_image_name = os.getenv('docker_login') + '/' + os.getenv('image_tag') + \
'azcliazdevops'  + image_version_string


docker_file_path
aml_docker_image_name
# aml_sudo_docker_image_name
aml_azcli_docker_image_name
aml_azcli_azdevops_docker_image_name

docker_file_dir

working_path = docker_file_dir
! ls -l $working_path


'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/dockerfile_aml-sdk_docker_image_sdk.v1.0.76'

'georgedockeraccount/aml-sdk_docker_image:sdk.v1.0.76'

'georgedockeraccount/aml-sdk_docker_imageazcli:sdk.v1.0.76'

'georgedockeraccount/aml-sdk_docker_imageazcliazdevops:sdk.v1.0.76'

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build'

total 8
-rw-rw-r-- 1 loginvm0_1 loginvm0_1  321 Dec  3 03:26 aml_sdk_conda_dep_file.yml
-rw-rw-r-- 1 loginvm0_1 loginvm0_1 1840 Dec  3 03:26 dockerfile_aml-sdk_docker_image_sdk.v1.0.76


In [9]:
build_command = 'docker build -t ' + aml_docker_image_name  + \
 ' --target AzureMLSDKOnly -f ' + \
docker_file_path +' '+ working_path + ' --no-cache'
build_command

!     $build_command
!     docker build -t $aml_azcli_docker_image_name  --target AzureMLSDKAndAzCli -f $docker_file_path $working_path
!     docker build -t $aml_azcli_azdevops_docker_image_name -f $docker_file_path $working_path


'docker build -t georgedockeraccount/aml-sdk_docker_image:sdk.v1.0.76 --target AzureMLSDKOnly -f /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build/dockerfile_aml-sdk_docker_image_sdk.v1.0.76 /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_build --no-cache'

Sending build context to Docker daemon  4.608kB
Step 1/11 : FROM continuumio/miniconda3:4.7.12 as AzureMLSDKOnly
 ---> 406f2b43ea59
Step 2/11 : MAINTAINER George Iordanescu <ghiordan@microsoft.com>
 ---> Running in c696f69cac39
Removing intermediate container c696f69cac39
 ---> a163c49d5da2
Step 3/11 : RUN apt-get update --fix-missing && apt-get install -y --no-install-recommends     gpg-agent     gcc g++     wget bzip2     curl apt-transport-https lsb-release gpg     git make     sudo &&     apt-get clean &&     rm -rf /var/lib/apt/lists/*
 ---> Running in d81dca36dc9d
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://deb.debian.org/debian buster-updates InRelease [49.3 kB]
Get:3 http://deb.debian.org/debian buster/main amd64 Packages [7908 kB]
Get:4 http://security.debian.org/debian-security buster/updates InRelease [65.4 kB]
Get:5 http://security.debian.org/debian-security buster/updates/main amd64 Packages [159 kB]
Fetched 8303 kB in 1s (6187 kB/s)
Reading p

Selecting previously unselected package libmpdec2:amd64.
Preparing to unpack .../libmpdec2_2.4.2-2_amd64.deb ...
Unpacking libmpdec2:amd64 (2.4.2-2) ...
Selecting previously unselected package libpython3.7-stdlib:amd64.
Preparing to unpack .../libpython3.7-stdlib_3.7.3-2_amd64.deb ...
Unpacking libpython3.7-stdlib:amd64 (3.7.3-2) ...
Selecting previously unselected package python3.7.
Preparing to unpack .../python3.7_3.7.3-2_amd64.deb ...
Unpacking python3.7 (3.7.3-2) ...
Selecting previously unselected package libpython3-stdlib:amd64.
Preparing to unpack .../libpython3-stdlib_3.7.3-1_amd64.deb ...
Unpacking libpython3-stdlib:amd64 (3.7.3-1) ...
Setting up python3-minimal (3.7.3-1) ...
Selecting previously unselected package python3.
(Reading database ... 13209 files and directories currently installed.)
Preparing to unpack .../00-python3_3.7.3-1_amd64.deb ...
Unpacking python3 (3.7.3-1) ...
Selecting previously unselected package apt-transport-https.
Preparing to unpack .../01-apt-tra

Setting up libpython3.7-stdlib:amd64 (3.7.3-2) ...
Setting up gpg-agent (2.2.12-1+deb10u1) ...
Setting up binutils (2.31.1-16) ...
Setting up libgcc-8-dev:amd64 (8.3.0-6) ...
Setting up cpp (4:8.3.0-1) ...
Setting up libc6-dev:amd64 (2.28-10) ...
Setting up libpython3-stdlib:amd64 (3.7.3-1) ...
Setting up libstdc++-8-dev:amd64 (8.3.0-6) ...
Setting up python3.7 (3.7.3-2) ...
Setting up gcc-8 (8.3.0-6) ...
Setting up python3 (3.7.3-1) ...
running python rtupdate hooks for python3.7...
running python post-rtupdate hooks for python3.7...
Setting up gcc (4:8.3.0-1) ...
Setting up g++-8 (8.3.0-6) ...
Setting up lsb-release (10.2019051400) ...
Setting up g++ (4:8.3.0-1) ...
update-alternatives: using /usr/bin/g++ to provide /usr/bin/c++ (c++) in auto mode
Processing triggers for libc-bin (2.28-10) ...
Processing triggers for mime-support (3.62) ...
Removing intermediate container d81dca36dc9d
 ---> 4a81a60722e2
Step 4/11 : ADD aml_sdk_conda_dep_file.yml /tmp/aml_sdk_conda_dep_file.yml
 ---> 

Removing intermediate container 6209c190c8f3
 ---> 6fcf3141047e
Step 8/11 : ENV CONDA_AUTO_UPDATE_CONDA=false
 ---> Running in 70078f033478
Removing intermediate container 70078f033478
 ---> e8939376a05e
Step 9/11 : ENV CONDA_DEFAULT_ENV=$CONDA_ENV_NAME
 ---> Running in f205c411d4f1
Removing intermediate container f205c411d4f1
 ---> 70a489f26983
Step 10/11 : ENV CONDA_PREFIX=$CONDA_DIR/envs/$CONDA_DEFAULT_ENV
 ---> Running in 76340bd1e446
Removing intermediate container 76340bd1e446
 ---> 45055c5245d2
Step 11/11 : ENV PATH=$CONDA_PREFIX/bin:/opt/conda/bin:$PATH
 ---> Running in c5a06fbc1881
Removing intermediate container c5a06fbc1881
 ---> 1d9b58d5bfcf
Successfully built 1d9b58d5bfcf
Successfully tagged georgedockeraccount/aml-sdk_docker_image:sdk.v1.0.76
Sending build context to Docker daemon  4.608kB
Step 1/17 : FROM continuumio/miniconda3:4.7.12 as AzureMLSDKOnly
 ---> 406f2b43ea59
Step 2/17 : MAINTAINER George Iordanescu <ghiordan@microsoft.com>
 ---> Using cache
 ---> a163c49d5da

  1100K .......... .......... .......... .......... .......... 13%  369K 3s
  1150K .......... .......... .......... .......... .......... 14%  112M 2s
  1200K .......... .......... .......... .......... .......... 14%  140M 2s
  1250K .......... .......... .......... .......... .......... 15%  236M 2s
  1300K .......... .......... .......... .......... .......... 15%  239M 2s
  1350K .......... .......... .......... .......... .......... 16%  851K 2s
  1400K .......... .......... .......... .......... .......... 16%  173M 2s
  1450K .......... .......... .......... .......... .......... 17%  119M 2s
  1500K .......... .......... .......... .......... .......... 18%  208M 2s
  1550K .......... .......... .......... .......... .......... 18%  246M 2s
  1600K .......... .......... .......... .......... .......... 19%  130M 2s
  1650K .......... .......... .......... .......... .......... 19%  117M 2s
  1700K .......... .......... .......... .......... .......... 20%  156M 2s
  1750K ....

  5450K .......... .......... .......... .......... .......... 64% 1.74M 1s
  5500K .......... .......... .......... .......... .......... 65%  105M 1s
  5550K .......... .......... .......... .......... .......... 65%  237M 1s
  5600K .......... .......... .......... .......... .......... 66%  250M 0s
  5650K .......... .......... .......... .......... .......... 66%  239M 0s
  5700K .......... .......... .......... .......... .......... 67% 56.8M 0s
  5750K .......... .......... .......... .......... .......... 67%  145M 0s
  5800K .......... .......... .......... .......... .......... 68%  174M 0s
  5850K .......... .......... .......... .......... .......... 69%  227M 0s
  5900K .......... .......... .......... .......... .......... 69%  238M 0s
  5950K .......... .......... .......... .......... .......... 70% 1.22M 0s
  6000K .......... .......... .......... .......... .......... 70%  128M 0s
  6050K .......... .......... .......... .......... .......... 71%  225M 0s
  6100K ....

 ---> Running in a06f0690b1e8
Removing intermediate container a06f0690b1e8
 ---> 1eff6906c84a
Step 20/20 : CMD /bin/bash
 ---> Running in 5a892cca18fc
Removing intermediate container 5a892cca18fc
 ---> b4f95116dd87
Successfully built b4f95116dd87
Successfully tagged georgedockeraccount/aml-sdk_docker_imageazcliazdevops:sdk.v1.0.76


In [10]:
# import docker

# cli = docker.APIClient()
# response=''
# response = [line for line in cli.build(
#     path=working_path, 
#     dockerfile=docker_file_path, 
#     pull=True,
#     rm=True, 
#     quiet=False,
#     nocache=True,
#     tag=aml_docker_image_name
# )]
# response[100:]

# client = docker.from_env()
# client.containers.list()

In [11]:
local_dir = !pwd
crt_local_dir = os.path.join(*([local_dir.s, 'docker_run_dir02']))
crt_local_dir 
!mkdir -p $crt_local_dir
!rm -rf $crt_local_dir/test_*
!ls -l $crt_local_dir
# ! chmod -R ugo=rwx $crt_local_dir

'/datadrive01/prj/PowerAIWithDocker/amlsdk/docker_run_dir02'

total 8
-rwxrwxrwx 1 root root 5662 Dec  3 03:22 exported_conda_env.yml


In [12]:
# export the resulted conda env

cli_command_inside_container = ' /bin/bash -c "conda env list ; pwd; ls -l /workspace ; ' + \
    'python -c \'import azureml.core;print(azureml.core.VERSION)\'; ' + \
    'conda env export > /workspace/exported_conda_env.yml"'

cli_command='docker run -it --rm  --name aml-sdk_docker_container ' + \
'-v ' + crt_local_dir + ':/workspace:rw ' + \
aml_docker_image_name + \
cli_command_inside_container

cli_command

if 1:
    ! $cli_command
else:
    client = docker.from_env()
    client.containers.run(aml_docker_image_name, 
                          runtime='',
                          remove=True,
                          volumes={'/workspace': {'bind': '/workspace', 'mode': 'rw'}},
                          working_dir='/',
                          command=cli_command_inside_container)

'docker run -it --rm  --name aml-sdk_docker_container -v /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_run_dir02:/workspace:rw georgedockeraccount/aml-sdk_docker_image:sdk.v1.0.76 /bin/bash -c "conda env list ; pwd; ls -l /workspace ; python -c \'import azureml.core;print(azureml.core.VERSION)\'; conda env export > /workspace/exported_conda_env.yml"'

# conda environments:
#
base                     /opt/conda
aml-sdk-conda-env     *  /opt/conda/envs/aml-sdk-conda-env

/
total 8
-rwxrwxrwx 1 root root 5662 Dec  3 03:22 exported_conda_env.yml
1.0.76


In [13]:
saved_conda_dependency_file = 'aml_sdk_conda_dep_file'+os.getenv('image_version')+'.yml'
exported_conda_dependency_file = 'aml_sdk_conda_dep_file_exported'+os.getenv('image_version')+'.yml'

!mkdir -p ./docker_history
!cp $docker_file_path ./docker_history/
!cp $conda_dependency_file_path ./docker_history/$saved_conda_dependency_file
!cp {os.path.join(*([crt_local_dir, 'exported_conda_env.yml']))} ./docker_history/$exported_conda_dependency_file



### To run aml SDK, ssh into a vm, run the command below and then go to: 
your_vm.eastus2.cloudapp.azure.com:9001/  
  
Make sure the host port (9001) is open in the VM.  

In [14]:
!echo docker run -it -p 9001:8888 -v $(pwd)/../:/workspace:rw $aml_docker_image_name /bin/bash -c '"jupyter notebook --notebook-dir=/workspace --ip='*' --port=8888 --no-browser --allow-root"'

docker run -it -p 9001:8888 -v /datadrive01/prj/PowerAIWithDocker/amlsdk/../:/workspace:rw georgedockeraccount/aml-sdk_docker_image:sdk.v1.0.76 /bin/bash -c "jupyter notebook --notebook-dir=/workspace --ip=* --port=8888 --no-browser --allow-root"


### Run short tests
Run ' import azureml.core; print("AML SDK version:", azureml.core.VERSION) ' in plain python and in a notebook.
  
Compare the SDK version against latest AML SDK [relese notes](https://docs.microsoft.com/en-us/azure/machine-learning/service/azure-machine-learning-release-notes)

In [15]:
!echo docker run  $aml_docker_image_name /bin/bash -c "python -c 'import azureml.core;print(azureml.core.VERSION)'"
!docker run  $aml_docker_image_name /bin/bash -c "python -c 'import azureml.core;print(azureml.core.VERSION)'"

!docker run  $aml_azcli_docker_image_name /bin/bash -c "python -c 'import azureml.core;print(azureml.core.VERSION)'; az --version; azcopy -h"
!docker run  $aml_azcli_azdevops_docker_image_name /bin/bash -c "python -c 'import azureml.core;print(azureml.core.VERSION)'; az --version; az devops -h"


docker run georgedockeraccount/aml-sdk_docker_image:sdk.v1.0.76 /bin/bash -c python -c 'import azureml.core;print(azureml.core.VERSION)'
1.0.76
1.0.76
azure-cli                         2.0.77

command-modules-nspkg              2.0.3
core                              2.0.77
nspkg                              3.0.4
telemetry                          1.0.4

Python location '/opt/az/bin/python3'
Extensions directory '/root/.azure/cliextensions'

Python (Linux) 3.6.5 (default, Nov 22 2019, 05:30:55) 
[GCC 6.3.0 20170516]

Legal docs and information: aka.ms/AzureCliLegal



Your CLI is up-to-date.
AzCopy 10.3.2
Project URL: github.com/Azure/azure-storage-azcopy

AzCopy is a command line tool that moves data into and out of Azure Storage.
To report issues or to learn more about the tool, go to github.com/Azure/azure-storage-azcopy

The general format of the commands is: 'azcopy [command] [arguments] --[flag-name]=[flag-value]'.

Usage:
  azcopy [command]

Available Commands:
  bench       Pe

#### Do the same in a  notebook

In [16]:
test_notebook_base_name = 'test_aml_sdk_docker_image'
test_notebook_name = test_notebook_base_name+'.ipynb'

In [17]:
%%writefile $crt_local_dir/$test_notebook_name

{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 3,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "AML SDK version: 0.1.68\n"
     ]
    }
   ],
   "source": [
    "# Check core SDK version number\n",
    "import azureml.core\n",
    "\n",
    "print(\"AML SDK version:\", azureml.core.VERSION)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python [conda env:aml-sdk-conda-env]",
   "language": "python",
   "name": "conda-env-aml-sdk-conda-env-py"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.6.6"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}


Writing /datadrive01/prj/PowerAIWithDocker/amlsdk/docker_run_dir02/test_aml_sdk_docker_image.ipynb


In [18]:
!ls -l $crt_local_dir

total 12
-rwxrwxrwx 1 root       root       5662 Dec  3 03:31 exported_conda_env.yml
-rw-rw-r-- 1 loginvm0_1 loginvm0_1  982 Dec  3 03:31 test_aml_sdk_docker_image.ipynb


#### Run the notebook and see the new SDK version


In [19]:
!docker run -v $crt_local_dir:/workspace:rw $aml_docker_image_name /bin/bash -c "jupyter kernelspec list && jupyter nbconvert --ExecutePreprocessor.kernel_name=python3 --execute --to notebook  /workspace/test_aml_sdk_docker_image.ipynb"

Available kernels:
  python3    /opt/conda/envs/aml-sdk-conda-env/share/jupyter/kernels/python3
[NbConvertApp] Converting notebook /workspace/test_aml_sdk_docker_image.ipynb to notebook
[NbConvertApp] Executing notebook with kernel: python3
[NbConvertApp] Writing 982 bytes to /workspace/test_aml_sdk_docker_image.nbconvert.ipynb


In [20]:
output_file = test_notebook_base_name+'.nbconvert.ipynb'
!cat $crt_local_dir/$output_file

{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "AML SDK version: 1.0.76\n"
     ]
    }
   ],
   "source": [
    "# Check core SDK version number\n",
    "import azureml.core\n",
    "\n",
    "print(\"AML SDK version:\", azureml.core.VERSION)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python [conda env:aml-sdk-conda-env]",
   "language": "python",
   "name": "conda-env-aml-sdk-conda-env-py"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.7.5"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}


#### Push image to ([dockerhub](https://hub.docker.com/)) registry (optional step)

In [21]:
# !docker login -u=$docker_login -p=
!docker push $aml_docker_image_name
# !docker push $aml_sudo_docker_image_name
!docker push $aml_azcli_docker_image_name
!docker push $aml_azcli_azdevops_docker_image_name

The push refers to repository [docker.io/georgedockeraccount/aml-sdk_docker_image]

8d209a0c: Preparing 
14a04e0c: Preparing 
1f15cd63: Preparing 
cb249b79: Preparing 
190fd43a: Preparing 


8d209a0c: Pushing  1.778GB/2.93GBBAPushing  1.677GB/2.93GB

8d209a0c: Pushed   2.989GB/2.93GBPushing  1.857GB/2.93GBPushing  1.884GB/2.93GBPushing  2.086GB/2.93GBPushing  2.228GB/2.93GBsdk.v1.0.76: digest: sha256:4ebadb1da228735aef7e1ef499133bd6f4a7539df969da88a3a1ca7fdb2d431f size: 1586
The push refers to repository [docker.io/georgedockeraccount/aml-sdk_docker_imageazcli]

1b84f444: Preparing 
6711d342: Preparing 
8383ff5b: Preparing 
8d209a0c: Preparing 
14a04e0c: Preparing 
1f15cd63: Preparing 
cb249b79: Preparing 
190fd43a: Preparing 
8383ff5b: Pushed   496.9MB/467.2MBraccount/aml-sdk_docker_image sdk.v1.0.76: digest: sha256:ae53fbac412fba21d3986b020fe08b5586debb936bef5644aa06d0d6bd77d4f6 size: 2218
The push refers to repository [docker.io/georgedockeraccount/aml-sdk_docker_imageazcliazdevops]

bfe40d43: Preparing 
1b84f444: Preparing 
6711d342: Preparing 
8383ff5b: Preparing 
8d209a0c: Preparing 
14a04e0c: Preparing 
1f15cd63: Preparing 
cb249b79: Preparing 
190fd43a: Preparing 
fe40d43: Pushed   17.68MB/16.25MBraccount/aml-sdk_docker_ima

In [22]:
!jupyter nbconvert --to html createAMLSDKDocker.ipynb

[NbConvertApp] Converting notebook createAMLSDKDocker.ipynb to html
[NbConvertApp] Writing 379589 bytes to createAMLSDKDocker.html
